In [ ]:
import spiceypy as spice

In [ ]:
from planetpy import utils

In [ ]:
spice.furnsh("/Users/klay6683/Dropbox/NotPublic/spice/"
             "cosp_1000_040701_040701/cas_2004_v21_040701_040701.tm")

## Get epoch time

Time coords taken from image label for N1467345444_2.
My utils just transforms the NASA date with days of the year to an isoformat string.

In [ ]:
import datetime as dt
time = utils.nasa_datetime_to_iso('2004-183T03:37:05')
time

Now converting this to an epoch time using spice

In [ ]:
et = spice.utc2et(time)
et

Feeding this `et` into the `spkpos` function of spice, we get the position data of Cassini in the Saturn reference frame at the time when above image was taken:

In [ ]:
vertex, lt = spice.spkpos("CASSINI", et, "IAU_SATURN", 'LT', 'SATURN')

The `getfov` function takes only SPICE instrument id numbers, no string identifiers, so I need to convert it first:

In [ ]:
camid = spice.bodn2c("CASSINI_ISS_NAC")

Now, we get the coordinates of the FOV of ISS NAC in the coordinate system of the NAC camera.

In [ ]:
shape, frame, borvec, no, corners = spice.getfov(camid, 4, 1000, 1000)
print("Getting a {shape} in frame {frame} with a boresight vector of {borvec}"
      .format(shape=shape, frame=frame, borvec=borvec))

In [ ]:
corners

To transform this into the Saturn ring plane (i.e. simply the Saturn system with z pointing up), we need to create the rotation matrix first:

In [ ]:
rotmat = spice.pxform('CASSINI_ISS_NAC', 'IAU_SATURN', et)

Now let's create the ring plane, located at (0,0,0) in the Saturn frame with `z` pointing up. We will be using `spice` planes for this:

In [ ]:
normal = np.array([0.,0.,1.])
point = np.zeros(3)
plane = spice.nvp2pl(normal, point)

The `spice.inrypl` function finds the intersection of a ray with a plane, but first we need to apply the rotation matrix from above to receive the ray in Saturn coordinates. We then store the results of `inrypl` into the new `corner_coords` array:

In [ ]:
corner_coords = []
for corner in corners:
    direction = rotmat @ corner
    print("direction:", direction)
    nxpts, xpt = spice.inrypl(vertex, direction, plane)
    corner_coords.append(xpt)


In [ ]:
corner_coords

In [ ]:
%matplotlib inline

In [ ]:
x = [i[0] for i in corner_coords]
y = [i[1] for i in corner_coords]

import matplotlib

fig, ax = plt.subplots(figsize=(8,8))
ax.scatter(x,y)
ax.ticklabel_format(useOffset=False)
ax.set_xlim(40300, 41000)
ax.set_ylim(118200, 118900)

In [ ]:
corner_coords

Looking at all corners and calculate a resolution over the whole length (not very good, but simple):

In [ ]:
def calc_dist_between(v1, v2):
    return spice.vnorm(v1 - v2)

for t in [(0,1), (0,3), (1,2),(2,3)]:
    print(t)
    print(calc_dist_between(corner_coords[t[0]], corner_coords[t[1]])/1024)

In [ ]:
cormin, cormax = corners[1][0:2]

In [ ]:
print(cormin,cormax)

In [ ]:
coord_range = np.linspace(cormin, cormax, 1024)

In [ ]:
coord_range

In [ ]:
delta = coord_range[1] - coord_range[0]

In [ ]:
import datetime as dt
import spiceypy as spice
import matplotlib

def calc_dist_between(v1, v2):
    return spice.vnorm(v1 - v2)

class Resolutor(object):
    normal = np.array([0.,0.,1.])
    point = np.zeros(3)
    plane = spice.nvp2pl(normal, point)
    boresight = np.array([0., 0., 1.])
    
    def __init__(self, timestr):
        self.timestr = timestr
        self.time = utils.nasa_datetime_to_iso(timestr)
        self.et = spice.utc2et(time)
        self.vertex, self.lt = spice.spkpos("CASSINI", self.et, "IAU_SATURN", 'LT', 'SATURN')
        self.camid = spice.bodn2c("CASSINI_ISS_NAC")
        shape, frame, borvec, no, corners = spice.getfov(camid, 4, 1000, 1000)
        self.corners = corners
        print("Getting a {shape} in frame {frame} with a boresight vector of {borvec}"
              .format(shape=shape, frame=frame, borvec=borvec))
        cormin, cormax = corners[1][0:2]
        self.coord_range = np.linspace(cormin, cormax, 1024)
        self.delta = self.coord_range[1] - self.coord_range[0]
        self.rotmat = spice.pxform('CASSINI_ISS_NAC', 'IAU_SATURN', et)
        self.calc_ring_coords()
        self.plot_coords()
        
    def calc_one_coord(self, coord):
        dir = self.rotmat @ coord
        nxpts, xpt = spice.inrypl(self.vertex, dir, self.plane)
        return xpt
    
    def calc_ring_coords(self):
        coords = []
        for corner in self.corners:
            coords.append(self.calc_one_coord(corner))

        # add boresight intercept
        dir = rotmat @ self.boresight
        nxpts, xpt = spice.inrypl(self.vertex, dir, self.plane)
        coords.append(self.calc_one_coord(self.boresight))
        self.coords = coords
        
    def calc_one_resolution(self, corner):
        coord = self.calc_one_coord(corner)
        coord1 = self.calc_one_coord(corner + np.array([self.delta, 0., 0.]))
        coord2 = self.calc_one_coord(corner + np.array([0., self.delta, 0.]))
        diff1 = calc_dist_between(coord, coord1)
        diff2 = calc_dist_between(coord, coord2)
        return diff1, diff2
    
    def plot_coords(self):
        x = [i[0] for i in self.coords]
        y = [i[1] for i in self.coords]
        
        fig, ax = plt.subplots(figsize=(8,8))
        ax.scatter(x,y)
        ax.ticklabel_format(useOffset=False)
#         ax.set_xlim(40300, 41000)
#         ax.set_ylim(118200, 118900)
        plt.show()

tstr = '2004-183T03:37:05'
resor = Resolutor(tstr)


In [ ]:
resor.corners

In [ ]:
for corner in resor.corners:
    print(resor.calc_one_resolution(corner))

In [ ]:
print(resor.calc_one_resolution(np.array([0,0,1])))